<a href="https://colab.research.google.com/github/kpaulfran/sapceship_titanic/blob/main/sapceship_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
import numpy as np
set_config(display='diagram')

In [81]:
train = pd.read_csv('https://raw.githubusercontent.com/kpaulfran/sapceship_titanic/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/kpaulfran/sapceship_titanic/main/test.csv')

In [82]:
train.shape

(8693, 14)

1) Fuente de datos

Kaggle: https://www.kaggle.com/competitions/spaceship-titanic/data?select=train.csv

2) Breve descripción de datos

set of personal records recovered from the ship's damaged computer system.

3) ¿Cuál es el objetivo?

Predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. 

4) ¿Este es un problema de clasificación o regresión?

classifier

5) ¿Cuántas características hay?

14

6) ¿Cuántas filas de datos hay?

8693

7) ¿Qué retos, si es que hay alguno, prevén en la limpieza, exploración o modelización con este conjunto de datos?

Habra que realizar imputaciones en ciertas variables ya que superan el 1% de datos missing

In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [84]:
train.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [85]:
#Exploración de datos por cada columna

train['PassengerId'].unique() # Sera separa al final #Grupo/# de persona en el grupo para evaluar si influye al graficarlo

array(['0001_01', '0002_01', '0003_01', ..., '9279_01', '9280_01',
       '9280_02'], dtype=object)

In [86]:
train['HomePlanet'].value_counts() #8492 datos / 201 datos na

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [87]:
round(1-(train['HomePlanet'].value_counts().sum()/8693),2) #Los datos faltantes son menor al 1% por lo que estas filas seran eliminadas para este caso

0.02

In [88]:
train.dropna(subset=['HomePlanet'], inplace=True)
train.shape

(8492, 14)

In [89]:
train['CryoSleep'].value_counts().sum()/8492 #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'most_frequent', se elige imputar por que ya hemos eliminado parte de nuestros datos

0.9746820536975978

In [90]:
train['Cabin'].unique() #Será separada al final, para luego graficarlo

array(['B/0/P', 'F/0/S', 'A/0/S', ..., 'G/1499/S', 'G/1500/S', 'E/608/S'],
      dtype=object)

In [91]:
train['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [92]:
train['Destination'].value_counts()

TRAPPIST-1e      5765
55 Cancri e      1769
PSO J318.5-22     780
Name: Destination, dtype: int64

In [93]:
train['Destination'].isna().sum()/train['Destination'].value_counts().sum() #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'most_frequent', se elige imputar por que ya hemos eliminado parte de nuestros datos

0.02140967043541015

In [94]:
train['Age'].describe()

count    8315.000000
mean       28.828743
std        14.490024
min         0.000000
25%        19.000000
50%        27.000000
75%        38.000000
max        79.000000
Name: Age, dtype: float64

In [95]:
train['Age'].isna().sum()  #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'mean'

177

In [96]:
train['VIP'].isna().sum()  #Luego se va a realizar la imputacion de estos datos utilizando el metodo 'most_frequent', se elige imputar por que ya hemos eliminado parte de nuestros datos

200

In [97]:
train['VIP'].value_counts()

False    8098
True      194
Name: VIP, dtype: int64

In [98]:
#Columnas de gastos en el spaceship seran sumadas 'total_billed'

In [99]:
#Eliminaremos Name para este análisis

train.drop(columns=['Name'], inplace = True)

In [100]:
train['Transported'].isna().sum() #Datos completos en columna objetivo

0

In [101]:
train['Transported'].value_counts() #Datos balanceados para una correcta clasificacion

True     4275
False    4217
Name: Transported, dtype: int64

In [102]:
#Segunda fase de la exploración y limpieza:

#Separar datos de las columnas 'PassengerID' y 'Cabin'

train['GroupId'] = train['PassengerId'].str.slice(stop = 4)
train['GroupId']

0       0001
1       0002
2       0003
3       0003
4       0004
        ... 
8688    9276
8689    9278
8690    9279
8691    9280
8692    9280
Name: GroupId, Length: 8492, dtype: object

In [103]:
train['PersonNumber'] = train['PassengerId'].str.slice(start = 5)
train.drop(columns=['PassengerId'],inplace=True)
train['PersonNumber']

0       01
1       01
2       01
3       02
4       01
        ..
8688    01
8689    01
8690    01
8691    01
8692    02
Name: PersonNumber, Length: 8492, dtype: object

In [104]:
#Cabin = [deck,num,side]

train[['deck','num','side']] = train['Cabin'].str.split('/', expand = True)
train.drop(columns=['Cabin'],inplace=True)

train['side']

0       P
1       S
2       S
3       S
4       S
       ..
8688    P
8689    S
8690    S
8691    S
8692    S
Name: side, Length: 8492, dtype: object

In [105]:
#Crearemos la columna total_billed y eliminaremos las columnas utilizadas.

#sums=df.sum(axis=1)

train['total_billed'] = train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)
train.drop(columns=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'],inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8492 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8277 non-null   object 
 2   Destination   8314 non-null   object 
 3   Age           8315 non-null   float64
 4   VIP           8292 non-null   object 
 5   Transported   8492 non-null   bool   
 6   GroupId       8492 non-null   object 
 7   PersonNumber  8492 non-null   object 
 8   deck          8299 non-null   object 
 9   num           8299 non-null   object 
 10  side          8299 non-null   object 
 11  total_billed  8492 non-null   float64
dtypes: bool(1), float64(2), object(9)
memory usage: 804.4+ KB


In [106]:
train.shape

(8492, 12)

In [107]:
#Analizaremos las nuevas columnas

train['GroupId'].value_counts() #No se observan tendencias

9081    8
8796    8
8728    8
5885    8
4256    8
       ..
3465    1
3462    1
3461    1
3459    1
4622    1
Name: GroupId, Length: 6107, dtype: int64

In [108]:
#Analisis
true_filter = train['Transported'] == True 
train_grouped = train.loc[true_filter, :]
train_grouped = train.groupby(by=['GroupId']).count()
train_grouped[['Transported','VIP']].sort_values(by = ['Transported'],ascending=False)[:10] #Se observa una fuerte tendencia a que los pasajeron VIP sean los que sean transportados, por lo que utilizaremos esta información sobre el GroupId y el numero de persona en el grupo

,Transported,VIP
GroupId,,
9081,8,7
8796,8,8
8728,8,8
5885,8,7
4256,8,8
8988,8,8
4005,8,8
5133,8,8
0984,8,8


In [109]:
train.drop(columns=['GroupId', 'PersonNumber'], inplace = True)

In [110]:
#Analisis de columnas deck, num, side

true_filter = train['Transported'] == True 
train_grouped = train.loc[true_filter, :]
train_grouped = train.groupby(by=['deck']).count()
train_grouped[['Transported','VIP']].sort_values(by = ['Transported'],ascending=False)[:10] #Se va a mantener por que se observa una correlacion entre el deck y los que son transportados

,Transported,VIP
deck,,
F,2724,2656
G,2498,2433
E,853,838
B,766,754
C,734,715
D,468,459
A,252,246
T,4,4


In [111]:
true_filter = train['Transported'] == True 
train_grouped = train.loc[true_filter, :]
train_grouped = train.groupby(by=['num']).count()
train_grouped[['Transported','VIP']].sort_values(by = ['Transported'],ascending=False)[:10] #la variacion entre transportadl y VIP por num es minima, no lo utilizaremos

,Transported,VIP
num,,
82,28,27
97,21,20
176,21,21
86,20,19
19,20,19
56,20,20
230,20,20
65,19,19
123,19,18


In [112]:
train.drop(columns=['num'], inplace=True)

In [113]:
true_filter = train['Transported'] == True 
train_grouped = train.loc[true_filter, :]
train_grouped = train.groupby(by=['side']).count()
train_grouped[['Transported','VIP']].sort_values(by = ['Transported'],ascending=False)[:10] #no es considerable, se eliminara

,Transported,VIP
side,,
S,4186,4085
P,4113,4020


In [114]:
train.drop(columns=['side'], inplace=True)

In [115]:
train.head(5)

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,deck,total_billed
0,Europa,False,TRAPPIST-1e,39.0,False,False,B,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,True,F,736.0
2,Europa,False,TRAPPIST-1e,58.0,True,False,A,10383.0
3,Europa,False,TRAPPIST-1e,33.0,False,False,A,5176.0
4,Earth,False,TRAPPIST-1e,16.0,False,True,F,1091.0


In [116]:
train.isna().sum()

HomePlanet        0
CryoSleep       215
Destination     178
Age             177
VIP             200
Transported       0
deck            193
total_billed      0
dtype: int64

Para imputar los valores se van a utilizar las metodología:

- mean para valores numericos
- most frequent para valores categoricos

In [117]:
# Dividimo nuestro dataset
X = train.drop('Transported', axis=1)
y = train['Transported']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [118]:
numeric_pipe = make_pipeline(mean_imputer, scaler)
categorical_pipe = make_pipeline(freq_imputer, ohe)

In [119]:
# Tuples para Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb4947ef850>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb494816b50>)])

In [120]:
preprocessor.fit(X_train)


ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb4947ef850>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb494816b50>)])

In [121]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [122]:
# Comprueben los valores faltantes y que los datos se escalen y tengan una codificación one-hot
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (6369, 20)
